# Fun with Flags

This notebook shows you how to interpret SDSS-V targeting flags that are represented as a single column.

Andy Casey (andrew.casey@monash.edu)

You will need the `sdss_semaphore` package to easily interpret the SDSS flags. You can install it with:

```pip install sdss_semaphore```


In [1]:
import numpy as np
import os
from astropy.io import fits
from sdss_semaphore.targeting import TargetingFlags

# Let's load the Astra allStar file.
# At the time of writing (2023-10-11) this contains targeting flags for BOSS spectra (v6_1_1) and APOGEE DR17 (no APOGEE SDSS-V).
allStar = fits.open(os.path.expandvars("$MWM_ASTRA/0.4.4/summary/mwmAllStar-0.4.4.fits"))

In [2]:
# Let's look at the header information.
allStar[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
                                                                                
        METADATA                                                                
                                                                                
V_ASTRA = '0.4.4   '           / Astra version                                  
CREATED = '23-10-06 10:28:04'  / File creation time (UTC %y-%m-%d %H:%M:%S)     
                                                                                
        HDU DESCRIPTIONS                                                        
                                                                                
COMMENT HDU 0: Summary infor

In [3]:
# Let's play with flags for spectra observed by BOSS.
flags = TargetingFlags(allStar[1].data["SDSS5_TARGET_FLAGS"])

In [4]:
# Let's count the number of targets assigned to each carton (and skip empty cartons)
for label, count in flags.count(skip_empty=True).items():
    print(f"{label}: {count}")

ops_sky_apogee_0.1.0: 24
ops_sky_boss_0.1.0: 8314
ops_std_boss_tic_0.1.3: 2024
ops_std_eboss_0.5.0: 32272
ops_std_boss_0.5.0: 92657
ops_std_boss_red_0.5.0: 11631
ops_std_apogee_0.5.0: 4828
mwm_cb_gaiagalex_apogee_0.5.0: 119
mwm_cb_gaiagalex_boss_0.5.0: 30285
mwm_cb_cvcandidates_boss_0.5.0: 550
mwm_galactic_core_0.5.0: 3884
mwm_cb_uvex3_0.5.0: 496
mwm_cb_uvex4_0.5.0: 830
mwm_snc_100pc_boss_0.5.0: 23890
mwm_snc_250pc_apogee_0.5.0: 1713
mwm_snc_250pc_boss_0.5.0: 577
mwm_yso_disk_apogee_0.5.0: 5129
mwm_yso_disk_boss_0.5.0: 8593
mwm_yso_embedded_apogee_0.5.0: 19
mwm_yso_nebula_apogee_0.5.0: 1
mwm_yso_variable_apogee_0.5.0: 9234
mwm_yso_variable_boss_0.5.0: 10134
mwm_yso_cmz_apogee_0.5.0: 179
mwm_yso_cluster_apogee_0.5.0: 7786
mwm_yso_cluster_boss_0.5.0: 13657
mwm_rv_short_fps_0.5.0: 6
mwm_legacy_ir2opt_0.5.0: 18914
mwm_dust_core_0.5.0: 28
mwm_wd_core_0.5.0: 23739
mwm_tess_planet_0.5.0: 3964
mwm_snc_100pc_apogee_0.5.0: 1632
mwm_cb_uvex1_0.5.0: 9034
mwm_cb_uvex2_0.5.0: 22686
mwm_tessrgb_core_

# Flags have attributes

The counts above are for targets assigned to a specific carton, with a specific version. Sometimes we don't care about the carton version and we just want to count by the effective number of targets assigned to that carton (over all versions), or we might want to count by the number of targets assigned to each mapper, or some other attribute.BufferError

The `TargetingFlags.mapping` attribute contains a reference dictionary for all cartons, including their attributes. Here's how we can count targets by those attributes:


In [5]:
for mapper, count in flags.count_by_attribute("mapper").items():
    print(f"{mapper}: {count:,}")

mwm: 340,581
ops: 473,860
bhm: 311,903
open: 259,514


In [6]:
for program, count in flags.count_by_attribute("program").items():
    print(f"{program}: {count:,}")

mwm_snc: 47,419
mwm_cb: 51,501
mwm_halo: 21,973
mwm_yso: 56,822
mwm_rv: 766
mwm_ob: 98,649
ops_std: 234,635
mwm_wd: 26,603
mwm_gg: 0
mwm_planet: 6,115
bhm_aqmes: 32,289
bhm_filler: 163,877
mwm_tessrgb: 3,849
bhm_csc: 17,743
ops_sky: 180,496
bhm_rm: 92,919
bhm_spiders: 38,342
mwm_dust: 47
mwm_legacy: 18,978
mwm_galactic: 4,697
mwm_filler: 28,594
mwm_tess_ob: 0
mwm_erosita: 4,490
open_fiber: 259,514
commissioning: 62,273
SKY: 32,201
mwm_tessob: 0
mwm_magcloud: 0
mwm_bin: 3,488
mwm_validation: 1,809
mwm_monitor: 0


# I own a carton. Where are my spectra?

You can efficiently construct boolean masks to select spectra that are assigned to a carton, program, mapper, or any other attribute.

The easiest way to do this is with the following methods:
- `TargetingFlags.in_carton_pk`: If you know the database primary key (`targetdb.carton.pk`) of your carton
- `TargetingFlags.in_carton_name`: If you know the name of your carton.
- `TargetingFlags.in_mapper`: To select by mapper (e.g., BHM, MWM)
- `TargetingFlags.in_program`: To select by program name.
- `TargetingFlags.in_alt_name`: To select by alternative name, curated with thanks by Marina Koukel.
- `TargetingFlags.in_alt_program`: To select by alternative program name, curated with thanks by Marina Koukel.

If you're not sure what your carton's alternative name or program is, you might find these properties useful:

- `TargetingFlags.all_mappers`: a tuple of all unique mapper names;
- `TargetingFlags.all_programs`: a tuple of all unique program names;
- `TargetingFlags.all_names`: a tuple of all unique carton names;
- `TargetingFlags.all_alt_names`: a tuple of all unique alternative names;
- `TargetingFlags.all_alt_programs`: a tuple of all unique alternative program names;

or you can look this up from `TargetingFlags.mapping`.

In [14]:
print("Mappers:")
for mapper in flags.all_mappers:
    print(f"\t{mapper}")
    
print("Programs")
for program in flags.all_programs:
    print(f"\t{program}")

print("Cartons names")
for name in flags.all_names:
    print(f"\t{name}")

print("Alternative cartons names")
for alt_name in flags.all_alt_names:
    print(f"\t{alt_name}")

print("Alternative programs")
for alt_program in flags.all_alt_programs:
    print(f"\t{alt_program}")


Mappers:
	ops
	bhm
	open
	mwm
Programs
	bhm_csc
	mwm_yso
	mwm_dust
	bhm_rm
	mwm_tess_ob
	bhm_aqmes
	mwm_wd
	open_fiber
	mwm_erosita
	mwm_bin
	ops_sky
	commissioning
	mwm_validation
	ops_std
	mwm_galactic
	mwm_cb
	mwm_tessrgb
	mwm_tessob
	mwm_planet
	bhm_filler
	mwm_ob
	mwm_filler
	mwm_monitor
	SKY
	mwm_magcloud
	mwm_legacy
	bhm_spiders
	mwm_snc
	mwm_gg
	mwm_halo
	mwm_rv
Cartons names
	mwm_galactic_core_dist_apogee
	bhm_csc_boss_dark
	mwm_ob_core
	ops_std_apogee
	mwm_snc_100pc
	mwm_cb_gaiagalex_boss
	openfibertargets_nov2020_47e
	manual_mwm_halo_distant_kgiant_far_boss_single
	mwm_erosita_stars_boss
	mwm_halo_mp_xp_boss
	manual_bright_targets_g13_offset_fixed_7
	ops_std_boss_ps1dr2
	mwm_yso_variable_boss_single
	manual_mwm_planet_transiting_bd_apogee
	openfibertargets_nov2020_22
	ops_std_boss_lsdr8
	mwm_rv_short-rm
	mwm_yso_pms_boss_zari18pms_single
	manual_mwm_halo_vmp_wise_boss_single
	manual_mwm_halo_distant_kgiant_near_boss
	manual_mwm_halo_distant_bhb_boss
	mwm_snc_250pc_apogee
	op

## Select by Mapper

In [8]:
# Select things assigned to MWM
mwm_mask = flags.in_mapper("mwm")
mwm_mask

array([ True,  True, False, ..., False, False, False])

In [9]:
# The `mwm_mask` is a N-length boolean array, where N is the number of targets in the allStar file.
assert len(mwm_mask) == len(allStar[1].data)

In [10]:
print(f"There are {np.sum(flags.in_mapper('mwm')):,} things assigned to MWM")

There are 340,581 things assigned to MWM


## Select by Program

In [11]:
flags.in_program("mwm_wd")

array([False, False, False, ..., False, False, False])

## Select by name

In [12]:
flags.in_carton_name("mwm_snc_250pc")

array([False, False, False, ..., False, False, False])